In [6]:
import zipfile
import os
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import defaultdict
import ast
from tqdm.notebook import tqdm as tqdm
import pickle
import contractions

c:\Users\srija\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [7]:
def helper(inv_idx_freq, inv_idx_postings, set_of_tokens, docID):
    for j in set_of_tokens:
        inv_idx_postings[j].append(docID)
        inv_idx_freq[j] = inv_idx_freq[j] + 1
        
        

In [8]:
def t1_and_t2(posting1, posting2):
  
  # if any of the postings is empty then their intersection will also be an empty list
  if len(posting1) == 0 or len(posting2) == 0:
    return []

  ptr1 = 0 #pointer that iterates over posting 1
  ptr2 = 0 #pointer that iterates over posting 2

  comparisons = 0 #intialize number of comparison as 0
  ans = []


  while ptr1 < len(posting1) and ptr2 < len(posting2):
    comparisons += 1 
    if posting1[ptr1] ==  posting2[ptr2]: # this means that both the docs contains the word.
      ans.append(posting1[ptr1])
      ptr1 = ptr1 + 1
      ptr2 = ptr2 + 1

    elif posting1[ptr1] <  posting2[ptr2]:
      ptr1 = ptr1 + 1
    
    else:
      ptr2 = ptr2 + 1

  return ans, comparisons


In [9]:
def not_t1(posting1):
    global docIDs
    ans = [] # to store all the docIDs except that are their in postings1

    for i in docIDs:
        if i in posting1:
            continue
        else:
            ans.append(i)
    return ans
    


In [2]:
# def t1_and_not_t2(posting1, posting2):
#     # if the first posting is empty then the intersection will also be an empty list
#     if len(posting1) == 0: 
#         return [], 0

#     if len(posting2) == 0:
#         return posting1, len(posting1)
    
#     # perform not query for posting2
#     not_posting2 = not_t1(posting2)

#     # now perform the and operation for posting1 and not_posting2
#     ans, comparisons = t1_and_t2(posting1, not_posting2)
#     return ans, comparisons
    
def t1_and_not_t2(posting1, posting2):
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons += 1

        if posting1[ptr1] == posting2[ptr2]: 
            ptr1+=1
            ptr2+=1
            continue
        elif posting1[ptr1] < posting2[ptr2]:
            ans.append( posting1[ptr1] )
            ptr1+=1

        else:
            ptr2+=1

    while ptr1 < len(posting1):
        ans.append(posting1[ptr1])
        ptr1+=1

    return ans, comparisons

        

    




In [11]:
def t1_or_t2(posting1, posting2):

    #if any of the posting list is empty we will return the other list.
    if len(posting1) == 0 :
        return posting2, 0
    if len(posting2) == 0:
        return posting1, 0
        
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons+=1
        if posting1[ptr1] < posting2[ptr2]:
            ans.append(posting1[ptr1]) 
            ptr1+=1
        elif posting2[ptr2] < posting1[ptr1]  :
            ans.append(posting2[ptr2])
            ptr2+=1
        else:
            ans.append(posting2[ptr2])
            ptr1+=1
            ptr2+=1

    while(ptr1 < len(posting1)):
        ans.append( posting1[ptr1] )
        ptr1+=1
    
    while(ptr2 < len(posting2)):
        ans.append( posting2[ptr2] )
        ptr2+=1
    
    return ans, comparisons


        




In [12]:
def t1_or_not_t2(posting1, posting2):
    # here we need to take the not of posting2 and then do an or with posting 1

    if len(posting1) == 0: #if the first posting list is empty we will return the not of the other list.
        return not_t1(posting2)
    
    not_posting2 = not_t1(posting2)

    ans, comparisons = t1_or_t2(posting1, not_posting2)
    return ans, comparisons

    
    

In [13]:
# l1 = [1,2,3,4]
# l2 = [2,3,7,9,19]
# print(t1_or_not_t2(l1,l2))

In [14]:
inv_idx_freq  = defaultdict(int) # initializing the doc Frequency dictionary as a default dict which return 0 whenver a key does not exists.
inv_idx_postings  = defaultdict(list) # intializing the postings dictionary to return an empty list whenever it is invoked for a key that does not exits.
sample = 5
docIDs = []
folder_path = 'Q1_2_output'
for filename in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contents = file.read()
            docID = int(filename[-4:])
            docIDs.append(docID)
            # print(docID)
            # set_of_tokens = set(contents.strip('][').split(','))
            set_of_tokens = set(ast.literal_eval(contents))
            helper(inv_idx_freq, inv_idx_postings, set_of_tokens, docID)
            # print(f'{filename}: {contents}')
            # print(set_of_tokens)
            # sample -= 1
            # if (sample == 0):
            #     break


#sort the index in alphabetical order wrt terms
inv_idx_postings = dict(sorted(inv_idx_postings.items()))
inv_idx_freq = dict(sorted(inv_idx_freq.items()))

for key in inv_idx_postings:
    inv_idx_postings[key].sort() #sort the indivdual posting lists correponging to each term in the index


  0%|          | 0/1400 [00:00<?, ?it/s]

In [15]:
pickle.dump(inv_idx_postings, open('inv_idx_postings.pkl', 'wb'))
pickle.dump(inv_idx_freq, open('inv_idx_freq.pkl', 'wb'))

In [16]:
inv_idx_postings = pickle.load(open("inv_idx_postings.pkl", 'rb'))
inv_idx_freq = pickle.load(open("inv_idx_freq.pkl", 'rb'))

In [ ]:
print(inv_idx_postings)

In [18]:
print(inv_idx_freq['surface'])

286


# Query preprocessing

In [19]:
def remove_punc(token):
    translator = str.maketrans('', '', string.punctuation)
    return token.translate(translator)

In [20]:
def preprocess_query(query):
    content1 = query.lower()
    content1 = contractions.fix(content1)
    tokens = word_tokenize(content1)
    stop_words = set(stopwords.words("english"))
    tokens1 = [token for token in tokens if token not in stop_words]
    tokens2 = list(map(remove_punc, tokens1))
    tokens3 = [token for token in tokens2 if token.strip()]
    return tokens3

In [21]:
preprocess_query("experimental fly design")

['experimental', 'fly', 'design']

In [46]:
queries=int(input())
while(queries>0):
    query=input()
    query=preprocess_query(query)
    print(query)
    operations=input().split(',')
    print(operations)
    q_ptr=1
    o_ptr=0
    answer=inv_idx_postings[query[0]]
    comparisions=0
    while(q_ptr<len(query) and o_ptr<len(operations)):
        T2=query[q_ptr]
        operation=operations[o_ptr]
        if operation=="AND":
            answer,temp=t1_and_t2(answer, inv_idx_postings[T2])
            comparisions+=temp
        elif operation=="OR":
            answer,temp=t1_or_t2(answer, inv_idx_postings[T2])
            comparisions+=temp
        elif operation=="AND NOT":
            answer,temp=t1_and_not_t2(answer, inv_idx_postings[T2])
            comparisions+=temp
        elif operation=="OR NOT":
            answer,temp=t1_or_not_t2(answer, inv_idx_postings[T2])
            comparisions+=temp
        q_ptr+=1
        o_ptr+=1
        print(answer, comparisions)
    queries-=1

['investigation', 'experimental', 'wing']
['AND', 'AND']
[1, 19, 29, 30, 74, 78, 84, 170, 173, 176, 179, 184, 187, 188, 189, 197, 202, 207, 212, 216, 222, 225, 245, 251, 294, 339, 372, 423, 427, 442, 462, 497, 505, 522, 549, 552, 567, 569, 634, 635, 636, 662, 689, 694, 712, 713, 739, 766, 772, 801, 816, 836, 844, 858, 887, 907, 927, 932, 946, 959, 986, 996, 997, 1019, 1039, 1062, 1066, 1074, 1075, 1078, 1083, 1092, 1097, 1098, 1156, 1159, 1161, 1205, 1213, 1220, 1225, 1227, 1230, 1231, 1337, 1338, 1341, 1352, 1364] 441
[1, 30, 78, 189, 202, 222, 225, 442, 497, 636, 694, 712, 1062, 1074, 1075, 1092, 1337, 1338, 1341] 679


In [ ]:
inv_idx_postings['design']

[12,
 27,
 36,
 102,
 124,
 135,
 172,
 173,
 198,
 212,
 213,
 215,
 221,
 235,
 239,
 246,
 249,
 256,
 270,
 277,
 301,
 312,
 328,
 341,
 367,
 368,
 415,
 416,
 419,
 426,
 429,
 503,
 519,
 529,
 535,
 543,
 554,
 581,
 592,
 596,
 602,
 603,
 612,
 628,
 640,
 649,
 650,
 658,
 674,
 683,
 696,
 712,
 724,
 729,
 733,
 746,
 753,
 758,
 762,
 772,
 797,
 798,
 833,
 834,
 836,
 839,
 868,
 870,
 876,
 880,
 883,
 893,
 896,
 905,
 906,
 907,
 921,
 924,
 928,
 935,
 945,
 948,
 964,
 981,
 984,
 985,
 986,
 987,
 1012,
 1015,
 1018,
 1033,
 1039,
 1051,
 1059,
 1069,
 1093,
 1104,
 1117,
 1124,
 1131,
 1136,
 1176,
 1188,
 1207,
 1246,
 1279,
 1292,
 1293,
 1310,
 1342,
 1351,
 1353,
 1360,
 1362,
 1380,
 1387,
 1397,
 1399]